<a href="https://colab.research.google.com/github/jesus-anaya/ENSAYO/blob/master/CNN_THREE_CELLS_220610.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TO RUN IN COLAB, 3 CELLS NEEDED!
#usar cuenta: jesus.anaya.co para entrar a https://colab.research.google.com/
#D:\disco_D\anaya\Teledeteccion\CNN_NeuralNetworks
#last update by janaya 220610


#"""
#(1/3)
#Supporting script for the medium post titled:
#'Is CNN equally shiny on mid-resolution satellite data?'
#available at https://medium.com/p/9e24e68f0c08
#Author: Pratyush Tripathy
#Date: 29 May, 2021
#Following package versions were used:
#numpy - 1.17.2
#sklearn - 0.22.1
#pyrsgis - 0.3.9
#tensorflow - 2.0.0
#"""

#Go to CELL 1 to ensure enough RAM


#You need lot of RAM. Increase RAM (DEPRECATED) https://www.youtube.com/watch?v=MHTsS0XEx8E
## 1.Copy and Paste the link below to get a new CO page, 2.Save a copy in drive 3. Conectar (check RAM>25gb), 4. Insert a new code cell and go to #CELL 2

#Using this link (DEPRECATED)
#https://colab.research.google.com/drive/1GqZu6zmCy2vMNMZqO78DDuHvwv_9vJcp?usp=sharing


# CELL 1
#more CPU AND RAM: the following link can also increase GPU to 25GB (use the following link to increase RAM)
## 1.Copy and Paste the link below in the browser to get a new google CO page, 2.Save a copy in drive 3. RUN and Conectar (check RAM>25gb)
#https://colab.research.google.com/drive/1mx2SPcWU9UxUDUJfBbatLzpp8LIhmr0i?usp=sharing
#! lscpu # use this command to confirm 4 cores & check in connect the 25GB RAM

#this web page has the directions to the code
#https://towardsdatascience.com/is-cnn-equally-shiny-on-mid-resolution-satellite-data-9e24e68f0c08

#https://github.com/PratyushTripathy/Landsat-Classification-Using-Convolution-Neural-Network


#This is a video about tensor flow
#https://www.youtube.com/watch?v=tPYj3fFJGjk

# Copy and Paste the code below in a new cell before CELL 3

#CELL 2
# Select and copy and run all the code until #CELL 3

! lscpu

!pip install tensorflow==2.0.0
!pip install tensorflow-gpu==2.0.0
!pip install h5py==2.10.0
#!pip install keras==2.3.0
!pip install pyrsgis==0.3.9


import os, math, random, glob, time
random.seed(2)
import numpy as np
from pyrsgis import raster
from pyrsgis.ml import imageChipsFromFile
from sklearn.utils import resample

#%tensorflow_version 2.x
import tensorflow as tf
print(tf.version.VERSION)
print(tf.version)


from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from matplotlib import pyplot as plt


!pip list | grep tensorflow
!pip list | grep tensorflow-gpu
!pip list | grep Keras
!pip list | grep h5py
!pip list | grep tensorboard
!pip list | grep numpy
!pip list | grep sklearn
!pip list | grep pyrsgis
!python -V
#print(sklearn.__version__)
print(keras.__version__)

### PART - A: CREATING AND STORING IMAGE CHIPS AS NUMPY ARRAYS  ###

# View the working path
import os
from google.colab import drive
drive.mount('/content/gdrive')
 
#print(os.getcwd())

#path = "/content/gdrive/My Drive/NeuralNetworks"

# define the file names, size of these two files (rows, columns) MUST be equal
feature_file = '/content/gdrive/MyDrive/NeuralNetworks/B2B3B4B8B11B12_10M_cl_3.TIF'
label_file = '/content/gdrive/MyDrive/NeuralNetworks/AQ_32bit_ds_1.tif'

os.chdir("/content/gdrive/MyDrive/NeuralNetworks/TDS_CNN")

!pwd
!ls   

# create feature chips using pyrsgis
features = imageChipsFromFile(feature_file, x_size=5, y_size=5)# it uses lot of RAM, originally x_size=11, y_size=11

""" Update: 29 May 2021
Since I added this code chunk later, I wanted to make least 
possible changes in the remaining sections. The below line changes
the index of the channels. This will be undone at a later stage.
"""
features = np.rollaxis(features, 3, 1)

# read the label file and reshape it
ds, labels = raster.read(label_file)
labels = labels.flatten()

#
#code for reading and storing image...
#

# check for irrelevant values (we are interested in 1s and non-1s)
labels = (labels == 1).astype(int)

# print basic details
print('Input features shape:', features.shape)
print('Input labels shape:', labels.shape)
print('Values in input features, min: %d & max: %d' % (features.min(), features.max()))
print('Values in input labels, min: %d & max: %d' % (labels.min(), labels.max()))

# Go up one directory
os.chdir("/content/gdrive/MyDrive/NeuralNetworks/TDS_CNN")
!pwd

# Save the arrays as .npy files
np.save('CNN_7by7_features.npy', features) # uses lot of RAM, bit depth matters.
np.save('CNN_7by7_labels.npy', labels)
print('Arrays saved at location %s' % (os.getcwd()))
!ls

#""""
#(2/3)
import os, math, random, glob, time
random.seed(2)
import numpy as np
import tensorflow as tf
from sklearn.utils import resample
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Load arrays from .npy files
os.chdir("/content/gdrive/MyDrive/NeuralNetworks/TDS_CNN")
features = np.load('CNN_7by7_features.npy') # uses lot LOT! of RAM... more than 25GB
labels = np.load('CNN_7by7_labels.npy')

# Separate and balance the classes
built_features = features[labels==1]
built_labels = labels[labels==1]

unbuilt_features = features[labels==0]
unbuilt_labels = labels[labels==0]

print('Number of records in each class:')
print('Built: %d, Unbuilt: %d' % (built_labels.shape[0], unbuilt_labels.shape[0]))


# Downsample the majority class
unbuilt_features = resample(unbuilt_features,
                            replace = False, # sample without replacement
                            n_samples = built_features.shape[0], # match minority n
                            random_state = 2)

unbuilt_labels = resample(unbuilt_labels,
                          replace = False, # sample without replacement
                          n_samples = built_features.shape[0], # match minority n
                          random_state = 2)

print('Number of records in balanced classes:')
print('Built: %d, Unbuilt: %d' % (built_labels.shape[0], unbuilt_labels.shape[0]))

# Combine the balanced features
features = np.concatenate((built_features, unbuilt_features), axis=0)
labels = np.concatenate((built_labels, unbuilt_labels), axis=0)

# Normalise the features
features = features / 255.0
print('New values in input features, min: %d & max: %d' % (features.min(), features.max()))



# Define the function to split features and labels
def train_test_split(features, labels, trainProp=0.6):
    dataSize = features.shape[0]
    sliceIndex = int(dataSize*trainProp)
    randIndex = np.arange(dataSize)
    random.shuffle(randIndex)
    train_x = features[[randIndex[:sliceIndex]], :, :, :][0]
    test_x = features[[randIndex[sliceIndex:]], :, :, :][0]
    train_y = labels[randIndex[:sliceIndex]]
    test_y = labels[randIndex[sliceIndex:]]
    return(train_x, train_y, test_x, test_y)
  
# Call the function to split the data
train_x, train_y, test_x, test_y = train_test_split(features, labels)

#""" Update: 29 May 2021
#Transpose the features to channel last format.
#If you have commented out the rollaxis line in the 
#first place. You can comment the following two lines too.
#"""

# Transpose the features to channel last format
train_x = tf.transpose(train_x, [0, 2, 3, 1])
test_x = tf.transpose(test_x, [0, 2, 3, 1])

print('Reshaped split features:', train_x.shape, test_x.shape)
print('Split labels:', train_y.shape, test_y.shape)
_, rowSize, colSize, nBands = train_x.shape

# Create a model
model = keras.Sequential()
model.add(Conv2D(32, kernel_size=1, padding='valid', activation='relu', input_shape=(rowSize, colSize, nBands)))
model.add(Dropout(0.25))
model.add(Conv2D(48, kernel_size=1, padding='valid', activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Run the model
model.compile(loss='sparse_categorical_crossentropy', optimizer= 'rmsprop',metrics=['accuracy'])
history = model.fit(train_x, train_y, epochs=1)# change to 10 epochs when the code works.

# Predict for test data 
yTestPredicted = model.predict(test_x)
yTestPredicted = yTestPredicted[:,1]

# Calculate and display the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(test_y, yTestPredicted)
pScore = precision_score(test_y, yTestPredicted)
rScore = recall_score(test_y, yTestPredicted)
fScore = f1_score(test_y, yTestPredicted)

print("Confusion matrix:\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f, F-Score: %.3f" % (pScore, rScore, fScore))

# Save the model inside a folder to use later
if not os.path.exists(os.path.join(os.getcwd(), 'trained_models')):
     os.mkdir(os.path.join(os.getcwd(), 'trained_models'))


model.save('trained_models/220428_CNN_AQ_PScore%.3f_RScore%.3f_FScore%.3f.h5' % (pScore, rScore, fScore))

#Find this new model and replace the name in the "model" variable.

#""

#--------------------------------------------------------------------------------------------------------------------------------
# CELL 3
#(3/3)

import os, math, random
random.seed(2)
import numpy as np
import tensorflow as tf
from pyrsgis import raster
import glob, time
from pyrsgis.ml import imageChipsFromArray

!pwd
!ls   
os.chdir("/content/gdrive/MyDrive/NeuralNetworks")

# Load the saved model
# note that this model was built in step 2/2, it must be manually updated here.
#model = tf.keras.models.load_model('/content/gdrive/MyDrive/NeuralNetworks/TDS_CNN/trained_models/200415_CNN_Builtup_PScore0.950_RScore0.919_FScore0.934.h5')
model = tf.keras.models.load_model('/content/gdrive/MyDrive/NeuralNetworks/TDS_CNN/trained_models/220428_CNN_AQ_PScore0.946_RScore0.999_FScore0.972.h5')
!ls

# Load a new multispectral image
ds, featuresNewImage = raster.read('/content/gdrive/MyDrive/NeuralNetworks/B2B3B4B8B11B12_10M_cl_cl_cl.TIF')
outFile = 'AQ_CNN_predicted_7by7.tif'


# Generate image chips in the back-end, the size of the file can be different but it might run out of memory.
def CNNdataGenerator(mxBands, kSize):
    mxBands = mxBands / 255.0
    nBands, rows, cols = mxBands.shape
    margin = math.floor(kSize/2)
    mxBands = np.pad(mxBands, margin, mode='constant')[margin:-margin, :, :]

    features = np.empty((rows*cols, kSize, kSize, nBands))

    n = 0
    for row in range(margin, rows+margin):
        for col in range(margin, cols+margin):
            feat = mxBands[:, row-margin:row+margin+1, col-margin:col+margin+1]

            b1, b2, b3, b4, b5, b6 = feat
            feat = np.dstack((b1, b2, b3, b4, b5, b6))

            features[n, :, :, :] = feat
            n += 1
            
    return(features)

# Call the function to generate features tensor
new_features = CNNdataGenerator(featuresNewImage, kSize=5) # lo modifiqué a 5 basado en lo que hice arriba
print('Shape of the new features', new_features.shape)

# Predict new data and export the probability raster
newPredicted = model.predict(new_features)
newPredicted = newPredicted[:,1]

prediction = np.reshape(newPredicted, (ds.RasterYSize, ds.RasterXSize))
raster.export(prediction, ds, filename=outFile, dtype='float')

plt.imshow(prediction)
plt.show() 